#### solve the linearized dynamics of the cart-pull mechanism


The compexity of dynamic constraints are increased in an attempt to obtain a more acurate solution to the swing up problem. In this section the small angle assumtion will still be made to simplify the model.

$$ cos(q_2) \approx 1 $$
$$ sin(q_2) \approx q_2 $$

This simplification removes the trigonometric functions from the dynamic constraints and is usually considered valid for angles of less than 10 degrees about the angle of linearization.

The simplified dynamic constraints for the optimization are:
$$ \ddot q_1 = \frac{lm_2sin(q_2)\dot q_2^2 + u + m_2gcos(q_2)sin(q_2)}{m_1 + m_2(1-cos^2(q_2))} \approx \frac{lm_2q_2\dot q_2^2 + u + m_2gq_2}{m_1} $$
$$ \ddot q_2 = - \frac{lm_2cos(q_2)sin(q_2)\dot q_2^2 + ucos(q_2) + (m_1+m_2)gsin(q_2)}{lm_1 + lm_2(1-cos^2(q_2))} \approx - \frac{lm_2q_2\dot q_2^2 + ucos(q_2) + (m_1+m_2)gq_2}{lm_1} $$

Both of these constraints are non-linear and and not able to be expressed in a quadratic or second order cone form becasue of the $lm_2q_2\dot q_2^2$ terms in each of the equations. This forces us to approach this optimization problem with a non linear solver like "Ipopt".

In [ ]:
using JuMP, Gurobi

#define initial and final states, x
x0 = [0 0 0 0]' 
xf = [3 pi 0 0]'   #wind up at the same angle, but 


"""
linearize the dynamics of the cart pull mechanism through the small angle assumption -> first term
in taylor series expansion for sin(q) and cos(q)

inputs:
    x0 = [q₁ q₂ q₁dot q₂dot]'      #initial system state
    xf = [q₁ q₂ q₁dot q₂dot]'      #final system state
    tf = [sec]                     #time to get to final state
    dt = [sec]                     #time between collocation points

outputs:
    path = [4xk]                   #state matrix that specifies path 
"""
function SolvLinearizedCartPull(x0, xf, tf, dt)
    
    #define model
    m = Model(solver = GurobiSolver(OutputFlag=0))
    T = 0:dt:tf                   #time vector
    K = length(T)                 #number of collocation points
    @variable(m,    x[1:4,1:K])   #system state at discrete times
    @variable(m, xdot[1:4,1:K])   #derivative of system state at discrete times
    @variable(m,    u[1:1,1:K])   #control input to system along q1

    #boundry constraints - initial and final state
    @constraint(m, x[:,1] .== x0)
    @constraint(m, x[:,K] .== xf)
    
    #path constraints
    #from future import path constraints
    
    
    #dynamics (path) constraints, integral form
    
    #model constants
    l = 1         #length of arm
    m1 = 3        #mass of cart
    m2 = 1        #mass of pendulum point mass
    g = 9.81
    
    
    #let's try forward euler first!
#     """calculate xdot of t as a function of x, u"""
#     function xdotcalc(x,u,t)
#         return 1 #might run into scoping issues 
#     end
    
    for k in 1:K-1
        # fill out xdot vector with values from x,u,t
    
        #notes - linearized system with first term in taylor series, and neglected coreolis acceleration
        @constraint(m,xdot[1,k] == x[3,k])
        @constraint(m,xdot[2,k] == x[4,k])
        @constraint(m,xdot[3,k] == u[1,k] + m2*g*1*x[2,k] / m1) 
        @constraint(m,xdot[4,k] == u[1,k] + (m1 + m2)*g*x[2,k] / m1) 
        
        #add forward euler dynamics constraint
        @constraint(m, x[:,k+1] .== x[:,k] + dt*xdot[:,k])
    end

    # minimize 2-norm (THIS IS LEAST-SQUARES)
    @objective(m, Min, sum(u.^2))
    solve(m)
    
    control = getvalue(u)
    xopt = getvalue(x)
    
    return (control, xopt)
end


In [ ]:

(c,x) = SolvLinearizedCartPull(x0,xf,3,.01)

using PyPlot
figure(figsize=(12,4))
plot(c[1,:])
plot( x[1,:], "b.-", markersize=4)
plot(x[2,:])

In [ ]:
x

In [ ]:
using JuMP, Gurobi, Ipopt

#define initial and final states, x
x0 = [0 pi 0 0]' 
xf = [3 pi 0 0]'   #wind up at the same angle, but 


"""
linearize the dynamics of the cart pull mechanism through the small angle assumption -> first term
in taylor series expansion for sin(q) and cos(q)

inputs:
    x0 = [q₁ q₂ q₁dot q₂dot]'      #initial system state
    xf = [q₁ q₂ q₁dot q₂dot]'      #final system state
    tf = [sec]                     #time to get to final state
    h =  [sec]                     #time between collocation points

outputs:
    path = [4xk]                   #state matrix that specifies path 
"""
function SolvNonStaticCartPull(x0, xf, tf, h, itype = "forwardEuler")
    
    #define model
    #m = Model(solver = GurobiSolver(OutputFlag=0))
    m = Model(solver = IpoptSolver())
    T = 0:h:tf                   #time vector
    K = length(T)                 #number of collocation points
    @variable(m,    x[1:4,1:K])   #system state at discrete times
    @variable(m, xdot[1:4,1:K])   #derivative of system state at discrete times
    @variable(m,    u[1:1,1:K])   #control input to system along q1

    #boundry constraints - initial and final state
    @constraint(m, x[:,1] .== x0)
    @constraint(m, x[:,K] .== xf)
    
    #path constraints
    
    #q₁ limits
#     xlower = -1
#     xupper =  4
#     for k in 1:K
#         @constraint(m, xlower <= x[1,k])
#         @constraint(m, x[1,k] <= xupper)
#     end
    
#     #q₂ limits
#      q2lower =  .5
#      q2upper =  -.5
    q2lower =   pi + .5
    q2upper =   pi - .5
    for k in 1:K
        @constraint(m, q2lower <= x[2,k])
        @constraint(m, x[2,k] <= q2upper)
    end
    
    
    #dynamics (path) constraints, integral form
    
    #model constants
    l = 1         #length of arm
    m1 = 3        #mass of cart
    m2 = 1        #mass of pendulum point mass
    g = 9.81
    linAngle = pi #the q2 angle we linearize the system about 
    
    
    #setup the dynamics constraints linearized about q1 = 0
    if linAngle == 0
        for k in 1:K
            #notes - linearized system with first term in taylor series, and neglected coreolis acceleration
            @constraint(m,xdot[1,k] == x[3,k])
            @constraint(m,xdot[2,k] == x[4,k])
            @constraint(m,xdot[3,k] == (l*m2*x[2,k]*x[4,k]^2 + u[1,k] + m2*g*1*x[2,k]) / m1) 
            @constraint(m,xdot[4,k] == (-l*m2*1*x[2,k]*x[4,k]^2 + u[1,k]*1 + (m1 + m2)*g*x[2,k]) / l*m1) 

        end
    end
    
    if linAngle == pi
        for k in 1:K
            #notes - linearized system with first term in taylor series, and neglected coreolis acceleration
            @constraint(m,xdot[1,k] == x[3,k])
            @constraint(m,xdot[2,k] == x[4,k])
            @NLconstraint(m,xdot[3,k] == (l*m2*x[2,k]*x[4,k]^2 + u[1,k] + m2*g*1*x[2,k]) / m1) 
            @NLconstraint(m,xdot[4,k] == (-l*m2*1*x[2,k]*x[4,k]^2 + u[1,k]*1 + (m1 + m2)*g*x[2,k]) / l*m1) 

        end
    end

    if itype == "forwardEuler" 
        #add forward euler dynamics constraint
        for k in 1:K-1
            @constraint(m, x[:,k+1] .== x[:,k] + h*xdot[:,k])
        end
        
    elseif itype == "backwardEuler"
        for k in 1:K-1
             #add backwards euler dynamics constraint
            @constraint(m, x[:,k+1] .== x[:,k] + .5*h*(xdot[:,k] + xdot[:,k+1]))
        end     
    end

    # minimize 2-norm (THIS IS LEAST-SQUARES)
    #@objective(m, Min, .5*h*sum(u[1:end-1].^2 + u[2:end].^2))
    @objective(m, Min, sum(u.^2) ) #+ sum(x[2,:]).^2))
    solve(m)
    #print(m)
    
    control = getvalue(u)
    xopt = getvalue(x)
    xdotopt = getvalue(xdot)
    
    return (control, xopt,xdotopt)
end

In [ ]:
(c,x) = SolvNonStaticCartPull(x0,xf,3,.01)

using PyPlot
figure(figsize=(12,4))
plot(c[1,:])
plot( x[1,:], "b.-", markersize=4)
plot(x[2,:])

#plot( x[1,:], x[2,:], "b.-", markersize=4 )
#plot( x[1,:][T], x[2,:][T], "b.", markersize=12 )
#plot( xw[1,:], xw[2,:], "r.", markersize=12 )
#axis("equal")
#axis((1.,8.,-.5,3.5));

In [ ]:
c

below, I define a library function which will convert the kinematics from a minimal set of generalized coordinates, to the maximal set that is required for my 3d ploting function:

In [97]:
"""takes a 3x3 rotation matrix and converts it to a 4x1 array of euler parameters"""
function A2P(A::Array) 
  e0 = sqrt((trace(A) + 1)/4)
  if e0 != 0
    e1 = (A[3,2] - A[2,3])/(4*e0)
    e2 = (A[1,3] - A[3,1])/(4*e0)
    e3 = (A[2,1] - A[1,2])/(4*e0)
  end

  if e0 == 0  #implies Χ = π
    #figure out which e terms are non-zero
    e1flag = false; e2flag = false; e3flag = false;
    if A[1,1] + 1 != 0  e1flag = true end
    if A[2,2] + 1 != 0  e2flag = true end
    if A[3,3] + 1 != 0  e3flag = true end

    if e1flag
      e1 = sqrt((A[1,1] + 1)/2)
      e2 = (A[2,1] + A[1,2])/(4*e1)
      e3 = (A[3,1] + A[1,3])/(4*e1)
    elseif e2flag
      e2 = sqrt((A[2,2] + 1)/2)
      e1 = (A[2,1] + A[1,2])/(4*e2)
      e3 = (A[3,2] + A[2,3])/(4*e2)
    elseif e3flag
      e3 = sqrt((A[3,3] + 1)/2)
      e1 = (A[3,1] + A[1,3])/(4*e3)
      e2 = (A[3,2] + A[2,3])/(4*e3)
    else
      warn("something is wrong with A value  $A ")
    end
  end

  p = [e0 e1 e2 e3]'
end

#principle rotations
Rx(Θ) = [1 0 0 ; 0 cos(Θ) -sin(Θ) ; 0 sin(Θ) cos(Θ)]
Ry(Θ) = [ cos(Θ) 0 sin(Θ) ; 0 1 0 ; -sin(Θ) 0 cos(Θ)]
Rz(Θ) = [ cos(Θ) -sin(Θ) 0 ; sin(Θ) cos(Θ) 0  ; 0 0 1]


Rz (generic function with 1 method)

In [130]:
using DataFrames
"""
convert state vector x for the cart pull problem from a minimal set of generalized coordinates to a
maximal set of cartesian coordinates using euler parameters to describe rotation. additionally, 
rotations will be made so that things look right in unity

inputs:
    x - state vector resulting from from our control optimization
    fln - string of file location
outputs:
    csv - write a csv to the proper location
    
assumptions:
-This code assumes the original simulation is resampled at 30Hz so the visualization timing is correct
- left handed coordinates as defined in unity:
    * y is up
    * x is left
    * z is out of the page

movement of the cart occures along the x, and rotation of the pendulum occures about z axis
"""
function process2CSV(x,path = "./pullCartVis3D/Assets/Data/data.csv")
    
    #extract state, we only need q1 and q2
    x = x[1:2,:]
    
    #build out fill state matrix  xFull = [r₁;r₂;p₁;p₂]
    nb = 2;
    xFull = zeros(7*nb, size(x)[2])  
    
    #place location parameters appropriately
    xFull[1,:] = x[1,:]
    xFull[4,:] = x[1,:]
    xFull[6,:] = .15*ones(size(x)[2],1)                 #unity compensation
    
    #place orientation parameters appropriately
    for i in range(1,size(x)[2])  
        xFull[7:10,i]   = A2P(Rx(pi/2)*eye(3))          #unity compensation
        xFull[11:14,i] = A2P(Rx(pi/2)*Ry(x[2,i]))       #unity compensation
    end
        
    #print to CSV
    xFull = convert(DataFrame, xFull)
    writetable(path,xFull)
end
    

#future:
"""
upsample or downsample 
"""
function resamplex(x,freq = 100);
    
    


process2CSV

In [131]:
process2CSV(x)

notes:

**what we have working:**
* a forward euler based solver with highly linearized dynamics, that is capable of solving 
    - optimal control of a non-inverted pendulum
    - optimal control of a swing up problem. 
    
* a unity visualizer that will display any movement of the pendulum, when given an input vector x


**Things we still need to do:**
- a proper introduction to for all these sections...
- this includes a proper derivation of the dynamics of the cart pull system
- explore what happens if we put simple path constraints on the existing system
- implement full direct collocation (with backwards euler / trapozoid method) 

**our modeling framework: levels of complexity and assumptions - each assumption removes a level of complexity in the dynamics**    
1. assumptions - small angles and quazi-static
   model - quadratic objective with Linear constraints (this is what we have working)
2. assumptions - small angles
   model - quadratic objective with quadratic constraints(to be implemented) - should solve with gurobi
3. no assumptions - full dynamics 
   model - quadratic objective with non-linear constraints - NLP - requires IPOPT library to solve


**order of integration method**
0. zero order
1. 1st order (trapezoid)
2. 2nd order or higher (math gets pretty intense)


**Implement a numerical solver to solve the system as an IVP** - I think that using a numerical solver to take the u(t) calculated by the method and then forward integrating it with the Real dynamics would be an interesting way to assess how bad each of the assumptions actually impacts the performance of this simple system. 
        
        
        
**Things we could do:**
    *explore simpsons-hermite transcription (higher order method than trapozoid)
    *explore the effect of ZOH, since this is what is actually possible as a control signal
    *explore exotic path constraints (swing up, but avoid these obsticles while you go through
    *alternative objective, robust control

    

**proposed organization of report:**

* Introduction
```julia
for model in increasing complexity```
    * Mathematical Model
    * Solution
    * Results / discussion
```julia
end```
* Conclusion
